In [4]:
import pandas as pd
from geopy import distance
import ast

neighb = pd.read_csv("neighbors.csv")
knn = pd.read_csv('knn.csv')
coord = pd.read_csv('cfips_location.csv')
neighb = pd.merge(neighb, coord, on='cfips')
neighb = neighb.drop(columns='cfips.1')
neighb['neighbors_cfips'] = neighb.apply(lambda x: ast.literal_eval(x.neighbors_cfips), axis=1)
neighb = neighb.rename(columns={'neighbors_cfips': 'neighbors'})

# Create a dictionary with cfips as keys and knn neighbors as values
knn_dict = knn.set_index('cfips')[['knn_1', 'knn_2', 'knn_3']].T.to_dict('list')

# Function to replace empty neighbors list with knn values
def replace_empty_neighbors(neighbors, cfips):
    if len(neighbors) == 0:
        return knn_dict[cfips]
    return neighbors

neighb['neighbors'] = neighb.apply(lambda row: replace_empty_neighbors(row['neighbors'], row['cfips']), axis=1)

def calculate_distances(x, neighbours, neighb_df):
    distances = []
    for cfips in neighbours:
        distance_value = round(
            distance.distance(
                neighb_df.loc[neighb_df.cfips == cfips, ['lat', 'lng']].values,
                x[['lat', 'lng']]
            ).miles
        )
        distances.append(distance_value)
    return distances


neighb['distances'] = neighb.apply(lambda x: calculate_distances(x, x['neighbors'], neighb), axis=1)
neighb['original_cfips'] = neighb['cfips']
neighb['cfips'] = range(0, len(neighb))

replace_dict = {}
for index, row in neighb.iterrows():
    replace_dict[row['original_cfips']] = row['cfips']
for row in range(len(neighb)):
    n_list = []
    for n in range(len(neighb.loc[row, 'neighbors'])):
        if neighb.loc[row, 'neighbors'][n] in replace_dict.keys():
            n_list.append(replace_dict[neighb.loc[row, 'neighbors'][n]])
    neighb.loc[row, 'neighbors'] = str(n_list)

neighb.to_csv('gnn_neighbor.csv')